In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
attractions = pd.read_csv(r"C:\Users\Bogere\OneDrive\Desktop\Tourism\tourism_attractions.csv")
Preferences = pd.read_csv(r"C:\Users\Bogere\OneDrive\Desktop\Tourism\userA_preferences.csv")

In [4]:
Attractions=attractions[['item_id','name','experience_tags']]
Attractions

,item_id,name,experience_tags
0,1,Bwindi Forest National Park,"Gorilla trekking, terrestrial animals, water s..."
1,2,Queen Elizabeth National Park,"Terrestrial animals, aquatic animals, beautifu..."
2,3,Murchison Falls National Park,"Terrestrial animals, water sports"
3,4,Source of the Nile River,"Water sports, beautiful scenery, cultural inst..."
4,5,Kibale Forest National Park,"Terrestrial animals, bird watching"
5,6,Kidepo Valley National Park,"Terrestrial animals, mountain sports, beautifu..."
6,7,Rwenzori Mountains National Park,"Terrestrial animals, mountain sports, beautifu..."
7,8,Lake Victoria,"Beautiful scenery, water sports, bird watching..."
8,9,Lake Bunyonyi,"Water sports, beautiful scenery, indigenous cu..."
9,10,Kampala Capital City,"Beautiful scenery, indigenous culture, cultura..."


In [5]:
Preferences

,user_id,preferences
0,1,"Terrestrial animals, aquatic animals, mountain..."


In [6]:
import nltk
from numpy import vectorize
nltk.download('stopwords')
stop_words=nltk.corpus.stopwords.words('english')
def normalize_document(document):
    document = re.sub(r'[^a-zA-Z0-9\s]', '', document, re.I|re.A)
    document = document.lower()
    document = document.strip()
    tokens = nltk.word_tokenize(document)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    document = ' '.join(filtered_tokens)
    return document
normalize_corpus = vectorize(normalize_document)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bogere\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import nltk
nltk.download('punkt')
norm_corpus_attractions = normalize_corpus(list(Attractions['experience_tags']))
norm_corpus_preferences = normalize_corpus(list(Preferences['preferences']))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bogere\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
tfidf_matrix_attractions = tfidf_vectorizer.fit_transform(norm_corpus_attractions)
tfidf_matrix_preferences = tfidf_vectorizer.transform(norm_corpus_preferences)

In [9]:
cosine_similarity = cosine_similarity(tfidf_matrix_attractions, tfidf_matrix_preferences)
df_cosinesimilarity = pd.DataFrame(cosine_similarity)
df_cosinesimilarity.index = df_cosinesimilarity.index +1
df_cosinesimilarity.index.name = 'item_id'
df_cosinesimilarity = df_cosinesimilarity.rename(columns={0: 'similarity_score'})
df_cosinesimilarity

,similarity_score
item_id,
1,0.150245
2,0.614907
3,0.318366
4,0.044601
5,0.270621
6,0.574915
7,0.608965
8,0.040632
9,0.047855


In [10]:
Attractions_with_similarity_scores = pd.merge(Attractions, df_cosinesimilarity, on='item_id')
similarity=Attractions_with_similarity_scores
similarity

,item_id,name,experience_tags,similarity_score
0,1,Bwindi Forest National Park,"Gorilla trekking, terrestrial animals, water s...",0.150245
1,2,Queen Elizabeth National Park,"Terrestrial animals, aquatic animals, beautifu...",0.614907
2,3,Murchison Falls National Park,"Terrestrial animals, water sports",0.318366
3,4,Source of the Nile River,"Water sports, beautiful scenery, cultural inst...",0.044601
4,5,Kibale Forest National Park,"Terrestrial animals, bird watching",0.270621
5,6,Kidepo Valley National Park,"Terrestrial animals, mountain sports, beautifu...",0.574915
6,7,Rwenzori Mountains National Park,"Terrestrial animals, mountain sports, beautifu...",0.608965
7,8,Lake Victoria,"Beautiful scenery, water sports, bird watching...",0.040632
8,9,Lake Bunyonyi,"Water sports, beautiful scenery, indigenous cu...",0.047855
9,10,Kampala Capital City,"Beautiful scenery, indigenous culture, cultura...",0.000000


In [11]:
similarity = similarity.sort_values(by='similarity_score', ascending=False)
recommendations_list = similarity['name'].tolist()
preference_list = ['indigenous culture', 'bird watching', 'mountain sports']
similarity

,item_id,name,experience_tags,similarity_score
1,2,Queen Elizabeth National Park,"Terrestrial animals, aquatic animals, beautifu...",0.614907
11,12,Lake Mburo National Park,"Terrestrial animals, aquatic animals, beautifu...",0.614907
6,7,Rwenzori Mountains National Park,"Terrestrial animals, mountain sports, beautifu...",0.608965
5,6,Kidepo Valley National Park,"Terrestrial animals, mountain sports, beautifu...",0.574915
14,15,Ziwa Rhino Sanctuary,"Terrestrial animals, aquatic animals, beautifu...",0.519217
2,3,Murchison Falls National Park,"Terrestrial animals, water sports",0.318366
13,14,Mount Elgon National Park,"Mountain sports, beautiful scenery, bird watching",0.313806
4,5,Kibale Forest National Park,"Terrestrial animals, bird watching",0.270621
12,13,Semuliki National Park,"Terrestrial animals, indigenous culture, bird ...",0.204572
10,11,Mgahinga Gorilla National Park,"Gorilla trekking, mountain sports, beautiful s...",0.163728


In [12]:
N = 5
top_5_recommendations = []
for index in range(0, N):
    top_5_recommendations.append(recommendations_list[index])

In [13]:
import pickle
from sklearn. neighbors import NearestNeighbors as knn
pickle.dump(knn,open('similarity1.pkl','wb'))

In [14]:
model = pickle.load(open('similarity1.pkl','rb'))
print(*('{}. {}'.format(*k) for k in enumerate(top_5_recommendations, start = 1)), sep="\n")

1. Queen Elizabeth National Park
2. Lake Mburo National Park
3. Rwenzori Mountains National Park
4. Kidepo Valley National Park
5. Ziwa Rhino Sanctuary
